<a href="https://colab.research.google.com/github/raynerz/niche_research/blob/main/Niche_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/googleapis/google-api-python-client
!pip install google-api-python-client
import pandas as pd

pd.set_option('max_colwidth', 1000)

# Build Youtube API


In [2]:
from googleapiclient.discovery import build

api_key='Your API Key Here' #TODO

youtube = build('youtube', 'v3', developerKey=api_key)

# Functions

## 1. Retrieve video ID's from a youtube search query

In [3]:
def get_video_ids(youtube, query, published_after, region_code, video_duration, max_results):
    """
    Retrieve video IDs from YouTube based on a specified search query and filtering parameters.

    Parameters:
    - youtube: The YouTube Data API service obtained by calling build('youtube', 'v3', developerKey=api_key).
    - query: A string representing the search query for which videos are to be retrieved.
    - published_after: A string representing the date and time in ISO 8601 format to filter videos published after this time.
    - region_code: A string representing the region code to focus the search results on a specific region.
    - videoDuration: A string representing the duration filter for videos, such as 'any', 'long', 'medium', or 'short'.
    - maxResults: An integer representing the maximum number of video IDs to retrieve.

    Returns:
    - A list of video IDs obtained from the search results.
    """
    # Set parameters for the search
    response = youtube.search().list(
        q=query,
        part='snippet',
        type="video",
        order="viewCount",
        publishedAfter=published_after,
        regionCode=region_code,
        videoDuration=video_duration,
        maxResults=max_results
    ).execute()

    # Process the search results
    video_ids = []
    for item in response['items']:
        if item['id']['kind'] == 'youtube#video':
            video_ids.append(item['id']['videoId'])

    return video_ids


## 2. Retrieve detailed information of the IDs you just searched for

In [14]:
def get_video_info(youtube, video_ids):
    """
    Retrieve detailed information for a list of YouTube videos based on their video IDs.

    Parameters:
    - youtube: The YouTube Data API service obtained by calling build('youtube', 'v3', developerKey=api_key).
    - video_ids: A list of strings containing the video IDs for which information is to be retrieved.

    Returns:
    - A list of dictionaries containing detailed information for each video, including its ID, title, duration in ISO 8601 format, and view count.
    """

    # Convert list of video IDs to a comma-separated string
    video_ids_str = ','.join(video_ids)

    # Make a request to the videos.list endpoint with the video IDs, 'snippet', 'contentDetails', and 'statistics' parts
    video_request = youtube.videos().list(
        part='snippet,contentDetails,statistics',
        id=video_ids_str
    )

    response = video_request.execute()

    video_info = []

    # Process the response
    for item in response['items']:
        video_id = item['id']
        title = item['snippet']['title']
        published_at = item['snippet']['publishedAt']
        duration_iso8601 = item['contentDetails']['duration']
        views = item['statistics']['viewCount'] if 'viewCount' in item['statistics'] else 'Not available'

        video_data = {
            'video_id': video_id,
            'video_link' : 'https://www.youtube.com/video/' + video_id,
            'title': title,
            'duration': duration_iso8601,
            'published_at': published_at,
            'views': views
        }

        video_info.append(video_data)

    return video_info

#Usage

Keep in mind that it has a daily limit of around 100 searches. [More info](https://developers.google.com/youtube/v3/guides/quota_and_compliance_audits)

In [15]:
video_ids = get_video_ids(youtube,
                          query="Space", #Your niche keywords here
                          published_after="2023-08-01T00:00:00Z",
                          region_code="US", #any country ISO codes e.g. US, GB, DE
                          video_duration="long", #"short", "medium", "long"
                          max_results=25) # Not more than 50

In [21]:
video_info = get_video_info(youtube, video_ids)
video_info = pd.DataFrame(video_info)

In [22]:

display(video_info)

,video_id,video_link,title,duration,published_at,views
0,uwekHBt4KIs,https://www.youtube.com/video/uwekHBt4KIs,Mystery of Apollo 13 Mission | Lost in Space | Dhruv Rathee,PT21M24S,2023-08-19T14:59:56Z,10597585
1,R18q4F6Lqnk,https://www.youtube.com/video/R18q4F6Lqnk,"Apollo Astronaut Breaks In Tears: ""The Moon Is NOT What You Think!""",PT21M16S,2023-08-22T16:41:40Z,7656384
2,1BwrXjgAPvw,https://www.youtube.com/video/1BwrXjgAPvw,Kalpana Chawla | Mystery of NASA Columbia Space Disaster | Dhruv Rathee,PT21M54S,2023-10-30T07:14:10Z,7107745
3,DcWqzZ3I2cY,https://www.youtube.com/video/DcWqzZ3I2cY,Jeff Bezos: Amazon and Blue Origin | Lex Fridman Podcast #405,PT2H11M32S,2023-12-14T17:53:21Z,2968026
4,uOI35G7cP7o,https://www.youtube.com/video/uOI35G7cP7o,Full Replay: SpaceX Launches Second Starship Flight Test,PT11H34M3S,2023-11-18T16:36:47Z,2812339
5,xOxq6cQwe7Y,https://www.youtube.com/video/xOxq6cQwe7Y,This Is What Happens When a Psychopath Is Unleashed on Space - Space Trash Scavenger | Ad,PT28M25S,2023-11-09T19:00:25Z,2239551
6,qDhP6lUO_AY,https://www.youtube.com/video/qDhP6lUO_AY,Anomalies in the Universe. Immersion in Deep Space,PT3H5M22S,2023-11-15T15:44:47Z,2120553
7,_P0ZK8xW8-4,https://www.youtube.com/video/_P0ZK8xW8-4,Космическая Гонка: Почему больше НЕ ЛЕТАЮТ на ЛУНУ? Подробная История!,PT3H19M52S,2023-12-19T21:54:09Z,1785626
8,nWm_OhIKms8,https://www.youtube.com/video/nWm_OhIKms8,The Future of Gaming: StarEngine (4K),PT24M27S,2023-11-22T18:00:10Z,1730111
9,34OJXQeIn64,https://www.youtube.com/video/34OJXQeIn64,What Is Beyond Edge Of The Universe?,PT1H34M31S,2023-09-24T13:35:06Z,1674343


#Save Results to Excel

In [23]:
video_info.to_excel('Niche_Research.xlsx')